# 실행 가능성 확인하기
### tensorflow 설치 시
- python 3.7 이하 버전 사용
- numpy 1.17 이하
- 64bit 환경에서 설치

### cv2 설치 시(PEP 오류 발생)
* ERROR: Could not build wheels for opencv-python which use PEP 517 and cannot be installed directly
* pip install --upgrade pip setuptools wheel (해결)

In [29]:
import cv2
import os
import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

In [6]:
EPOCHS = 10
DATA_PATH = 'data/2/'
DATA_OK_PATTERN = DATA_PATH + 'OK/*.png'
DATA_FAIL_PATTERN = DATA_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH = 'results/'

### Sequential 모델
각 레이어에 정확히 하나의 입력 텐서와 하나의 출력 텐서가 있는 일반 레이어 스택

In [4]:
def Model():
    return Sequential([Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
                      MaxPool2D(),
                      Conv2D(64, (3, 3), activation='relu'),
                      MaxPool2D(),
                      Conv2D(128, (3, 3), activation='relu'),
                      MaxPool2D(),
                      Conv2D(256, (3, 3), activation='relu'),
                      MaxPool2D(),
                      Flatten(),
                      Dense(1, activation='sigmoid')])

## 데이터셋 불러오기
- OK_PATTERN, FAIL_PATTERN 별로 구분
- preprocess 함수 -> 이미지 타입 변환

In [5]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1)
    return tf.image.convert_image_dtype(img, tf.float32)

In [21]:
ok_list = glob.glob(DATA_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0 for _ in range(len(ok_list))])

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = glob.glob(DATA_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1 for _ in range(len(fail_list))])

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train, Valid 데이터셋 나누기

In [25]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

In [26]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Epoch 1/10
22/22 [==============================] - 60s 3s/step - loss: 0.4388 - accuracy: 0.8407 - val_loss: 0.3807 - val_accuracy: 0.8900
Epoch 2/10
22/22 [==============================] - 61s 3s/step - loss: 0.3940 - accuracy: 0.8835 - val_loss: 0.3423 - val_accuracy: 0.8969
Epoch 3/10
22/22 [==============================] - 54s 2s/step - loss: 0.3674 - accuracy: 0.8835 - val_loss: 0.3599 - val_accuracy: 0.8797
Epoch 4/10
22/22 [==============================] - 50s 2s/step - loss: 0.3760 - accuracy: 0.8761 - val_loss: 0.3987 - val_accuracy: 0.8557
Epoch 5/10
22/22 [==============================] - 46s 2s/step - loss: 0.3516 - accuracy: 0.8805 - val_loss: 0.3325 - val_accuracy: 0.8969
Epoch 6/10
22/22 [==============================] - 45s 2s/step - loss: 0.3756 - accuracy: 0.8761 - val_loss: 0.4346 - val_accuracy: 0.8694
Epoch 7/10
22/22 [==============================] - 50s 2s/step - loss: 0.3798 - accuracy: 0.8761 - val_loss: 0.3558 - val_accuracy: 0.8866
Epoch 8/10
22/22 [==

In [31]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]

        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
        
        cv2.imwrite(path + '/%.4f_%04d.png' % (y, index), imgs[idx].numpy() * 255)
        index += 1